# Определение наиболее прибыльного региона добычи нефти

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import os
import numpy as np
from scipy import stats as st

# Оглавление

- [1. Введение](#1.)
- [1.1. Описание данных](#1.1.)
- [1.2. Описание проекта](#1.2.)
- [1.3. Условия задачи](#1.3.)
- [1.4. Исследование и подготовка данных](#1.4.)
- [2. Машинное обучение](#2.)
- [2.1. Прогнозирование объём запасов в скважинах региона 1](#2.1.)
- [2.2. Прогнозирование объём запасов в скважинах региона 2](#2.2.)
- [2.3. Прогнозирование объём запасов в скважинах региона 3](#2.3.)
- [2.4. Анализ результатов по всем регионам](#2.4.)
- [2.4.1. Регион 1](#2.4.1.)
- [2.4.2. Регион 2](#2.4.2.)
- [2.4.3. Регион 3](#2.4.3.)
- [2.4.4. Выводы](#2.4.4.)
- [3. Расчет прибыли](#3.)
- [3.1. Средний запас сырья](#3.1.)
- [3.2. Достаточный объём сырья для безубыточной разработки новой скважины](#3.2.)
- [3.3. Количество скважин для безубыточной добычи (дополнительный пункт. не из задания)](#3.3.)
- [4. Риски и прибыль для каждого региона](#4.)
- [4.1. Функции для проведения bootstrap и расчета прибыли](#4.1.)
- [4.1.1. Функция расчета прибыли](#4.1.1.)
- [4.1.2. Функция для проведения bootstrap](#4.1.2.)
- [4.2. Bootstrap](#4.2.)
- [4.3. Средняя прибыль](#4.3.)
- [4.4. Доверительный интервал](#4.4.)
- [4.5. Вероятность убытка](#4.5.)
- [5. Выводы](#5.)

# 1. Введение
<a id="1."></a>

## 1.1. Описание данных
<a id="1.1."></a>

**Признаки**
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).


# 1.2. Описание проекта
<a id="1.2."></a>
Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.
Шаги для выбора локации обычно такие:
- В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
- Строят модель для предсказания объёма запасов в новых скважинах;
- Выбирают скважины с самыми высокими оценками значений;
- Определяют регион с максимальной суммарной прибылью отобранных скважин.
Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.


# 1.3. Условия задачи
<a id="1.3."></a>
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.


In [2]:
path10 = 'geo_data_0.csv'
path11 = 'geo_data_1.csv'
path12 = 'geo_data_2.csv'
path20 = '/datasets/geo_data_0.csv'
path21 = '/datasets/geo_data_1.csv'
path22 = '/datasets/geo_data_2.csv'
if os.path.exists(path10) and os.path.exists(path11) and os.path.exists(path12):
    df_0 = pd.read_csv(path10)
    df_1 = pd.read_csv(path11)
    df_2 = pd.read_csv(path12)
else:
    df_0 = pd.read_csv(path20)
    df_1 = pd.read_csv(path21)
    df_2 = pd.read_csv(path22)

# 1.4. Исследование и подготовка данных
<a id="1.4."></a>

In [3]:
display(df_0.head(2))
display(df_0.info())
print('Количество задублированных записей в df_0:', df_0.duplicated().sum())
print('_'*60)
display(df_1.head(2))
display(df_1.info())
print('Количество задублированных записей в df_1:', df_1.duplicated().sum())
print('_'*60)
display(df_2.head(2))
display(df_2.info())
print('Количество задублированных записей в df_2:', df_2.duplicated().sum())
print('_'*60)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.22117,105.280062
1,2acmU,1.334711,-0.340164,4.36508,73.037750


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Количество задублированных записей в df_0: 0
____________________________________________________________


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261


...

Количество задублированных записей в df_1: 0
____________________________________________________________


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Количество задублированных записей в df_2: 0
____________________________________________________________


во всех датасетах наблюдается:
1. отсутсвие пропусков
2. отсутсвие задублированных записей
3. корректность типов данных

**вывод:** данные валидные. идем дальше.

В соответствии с заданием для обучения модели необходимо использовать линейную регрессию. Это значит, что возможно имеет смысл провести масштабирование данных.
Для этого необходимо:
1. Разбить данные на фичи и целевой признак
2. Разбить данные на обучающую и валидационную выборки
3. Провести масштабирование данных. Применим для этих целей StandardScaller.

прим. Модель необходимо опробовать как с учетом шага 3, так и без него.

In [4]:
# шаги 1,2
X_0_train, X_0_valid, y_0_train, y_0_valid = train_test_split(df_0.drop(columns=['product', 'id'], axis = 1), df_0.loc[:,'product'], random_state=42,  test_size=0.25)
X_1_train, X_1_valid, y_1_train, y_1_valid = train_test_split(df_1.drop(columns=['product', 'id'], axis = 1), df_1['product'], random_state=42, test_size=0.25)
X_2_train, X_2_valid, y_2_train, y_2_valid = train_test_split(df_2.drop(columns=['product', 'id'], axis = 1), df_2['product'], random_state=42, test_size=0.25)
# шаг 3
scaler = StandardScaler()
scaler.fit(X_0_train)
X_0_train_scaled = scaler.transform(X_0_train)
X_0_valid_scaled = scaler.transform(X_0_valid)

scaler.fit(X_1_train)
X_1_train_scaled = scaler.transform(X_1_train)
X_1_valid_scaled = scaler.transform(X_1_valid)

scaler.fit(X_2_train)
X_2_train_scaled = scaler.transform(X_2_train)
X_2_valid_scaled = scaler.transform(X_2_valid)

# 2. Машинное обучение
<a id="2."></a>

## 2.1. Прогнозирование объём запасов в скважинах региона 1
<a id="2.1."></a>

In [5]:
param_distributions_linreg = {
    'fit_intercept': [True, False],
    'normalize': [True, False],
    'copy_X': [True, False]}

linreg_model_0 = LinearRegression(n_jobs=-1)
#alpha_range = np.logspace(0.6, 2, 20)

randomsearch = RandomizedSearchCV(estimator=linreg_model_0, cv=5, param_distributions=param_distributions_linreg, random_state=42, n_iter=8, scoring='neg_root_mean_squared_error')
randomsearch.fit(X_0_train_scaled, y_0_train)

predicted_valid_0 = randomsearch.predict(X_0_valid_scaled)
mse_0 = mean_squared_error(y_0_valid, predicted_valid_0)
rmse_0 = mean_squared_error(y_0_valid, predicted_valid_0, squared=False)

print('best_params:', randomsearch.best_params_)
print("для региона 1 RMSE =", rmse_0)

best_params: {'normalize': True, 'fit_intercept': True, 'copy_X': True}
для региона 1 RMSE = 37.75660035026169


## 2.2. Прогнозирование объём запасов в скважинах региона 2
<a id="2.2."></a>

In [6]:
model_1 = LinearRegression(normalize=True, fit_intercept=True, copy_X=True)
model_1.fit(X_1_train_scaled, y_1_train)

predicted_valid_1 = model_1.predict(X_1_valid_scaled)
rmse_1 = mean_squared_error(y_1_valid, predicted_valid_1, squared=False)

print('для региона 2 RMSE =', rmse_1)

для региона 2 RMSE = 0.890280100102883


## 2.3. Прогнозирование объём запасов в скважинах региона 3
<a id="2.3."></a>

In [7]:
model_2 = LinearRegression(normalize=True, fit_intercept=True, copy_X=True)
model_2.fit(X_2_train_scaled, y_2_train)

predicted_valid_2 = model_2.predict(X_2_valid_scaled)
rmse_2 = mean_squared_error(y_2_valid, predicted_valid_2, squared=False)

print('для региона 3 RMSE =', rmse_2)

для региона 3 RMSE = 40.145872311342174


Изучим полученные результаты

## 2.4. Анализ результатов по всем регионам
<a id="2.4."></a>

Объединим по каждому региону истинные и спрогнозированные значения в один датафрейм и оценим визуально

## 2.4.1. Регион 1
<a id="2.4.1."></a>

In [8]:
true_predict_values_region_1 = pd.concat([y_0_valid.reset_index(drop=True), pd.DataFrame(predicted_valid_0)], axis = 1).rename(columns={'product': 'true', 0: 'predicted'})
true_predict_values_region_1.head(10)

,true,predicted
0,122.073350,101.901017
1,48.738540,78.217774
2,131.338088,115.266901
3,88.327757,105.618618
4,36.959266,97.980185
5,93.988227,80.915367
6,31.768403,81.485852
7,74.856619,94.663975
8,99.380950,81.526376
9,108.728916,99.389740


## 2.4.2. Регион 2
<a id="2.4.2."></a>

In [9]:
true_predict_values_region_2 = pd.concat([y_1_valid.reset_index(drop=True), pd.DataFrame(predicted_valid_1)], axis = 1).rename(columns={'product': 'true', 0: 'predicted'})
true_predict_values_region_2.head(10)

,true,predicted
0,0.000000,0.844738
1,53.906522,52.921612
2,134.766305,135.110385
3,107.813044,109.494863
4,0.000000,-0.047292
5,134.766305,136.356564
6,3.179103,2.660889
7,26.953261,26.796211
8,30.132364,30.318322
9,80.859783,81.733141


## 2.4.3. Регион 3
<a id="2.4.3."></a>

In [10]:
true_predict_values_region_3 = pd.concat([y_2_valid.reset_index(drop=True), pd.DataFrame(predicted_valid_2)], axis = 1).rename(columns={'product': 'true', 0: 'predicted'})
true_predict_values_region_3.head(10)

,true,predicted
0,117.441301,98.301916
1,47.841249,101.592461
2,45.883483,52.449099
3,139.014608,109.922127
4,84.004276,72.411847
5,60.527247,104.142542
6,61.429286,64.127656
7,60.632764,91.918582
8,135.749821,104.937986
9,104.381643,92.218485


## 2.4.4. Выводы
<a id="2.4.4."></a>

На втором датасете даже визуально наблюдается высокая точность прогнозов при этом видно, что присутствуют отрицательные значения. Заменим их на 0.

In [11]:
true_predict_values_region_2.loc[true_predict_values_region_2['predicted'] < 0, 'predicted'] = 0
true_predict_values_region_2.head()

,true,predicted
0,0.000000,0.844738
1,53.906522,52.921612
2,134.766305,135.110385
3,107.813044,109.494863
4,0.000000,0.000000


## 3. Расчет прибыли
<a id="3."></a>

## 3.1. Средний запас сырья
<a id="3.1."></a>

Вероятно в задании в п.2.4  имеется ввиду средний запас сырья **в регионе**.
Если так, расчитаем для каждого региона спрогнозированное и рельное среднее значение запасов.

In [12]:
display(true_predict_values_region_1.mean())
display(true_predict_values_region_2.mean())
display(true_predict_values_region_3.mean())

true         92.325956
predicted    92.398800
dtype: float64

true         68.725381
predicted    68.723322
dtype: float64

true         95.150999
predicted    94.771024
dtype: float64

Несмотря на достаточно большое значение RMSE среднее спрогнозированных и истинных значений выглядят очень похожими. Оценим расхождение в процентном соотнощении.

In [13]:
print('ошибка среднего значения для региона 1 составляет',
      abs(round((true_predict_values_region_1.mean()['true'] - true_predict_values_region_1.mean()['predicted']) / true_predict_values_region_1.mean()['true'] * 100, 3)), '%')
print('ошибка среднего значения для региона 2 составляет',
      abs(round((true_predict_values_region_2.mean()['true'] - true_predict_values_region_2.mean()['predicted']) / true_predict_values_region_2.mean()['true'] * 100, 3)), '%')
print('ошибка среднего значения для региона 3 составляет',
      abs(round((true_predict_values_region_3.mean()['true'] - true_predict_values_region_3.mean()['predicted']) / true_predict_values_region_2.mean()['true'] * 100, 3)), '%')

ошибка среднего значения для региона 1 составляет 0.079 %
ошибка среднего значения для региона 2 составляет 0.003 %
ошибка среднего значения для региона 3 составляет 0.553 %


**Вывод:** видно, что если прогнозировать среднее значение по региону, то чтоность достаточно высокая. Отлично. Переходим к расчету прибыли.

## 3.2. Достаточный объём сырья для безубыточной разработки новой скважины
<a id="3.2."></a>

На разработку 200 скважин в регионе по условию необходимо затратить 10 млрд рублей (10000 млн). Следовательно, разработка одной скважины обходится в 10000 млн / 200 шт.=  50 млн/шт.
При стоимости одного берреля в 450 р, в среднем одна скважина должна принести минимум 50 млн / 450 = 111111 баррелей. Округляя в большую сторону получим, что для безубыточной деятельности объем запасов одной средний скважины должен быть 111.2 тыс баррелей. Как видно, это выше, чем средний запас сырья по каждому из регионов (см. п.2.5.)

**Вывод:** средний запас сырья в регионе ниже чем расчетное количество для безубыточного производства. При таких ценах надо отбирать лучшие скважины.

## 3.3. Количество скважин для безубыточной добычи (дополнительный пункт. не из задания)
<a id="3.3."></a>

В п.3.2. мы посчитали объем нефти в скважине для безубыточной добычи. При этом средний объем скважины ниже этого количества. В соответствии с описанием из каждых 500 берут 200, т.е лучшие 40%. Это 60-й процентиль. Посмотрим какой объем сырья соответствует 60-му процентилю.

In [14]:
print('худшая из топ-200 скважин в первом регионе даст',  true_predict_values_region_1['true'].quantile(0.6), 'тыс баррелей')
print('худшая из топ-200 скважин во втором регионе даст', true_predict_values_region_2['true'].quantile(0.6), 'тыс баррелей')
print('худшая из топ-200 скважин в третьем регионе даст', true_predict_values_region_3['true'].quantile(0.6), 'тыс баррелей')

худшая из топ-200 скважин в первом регионе даст 106.29224770845815 тыс баррелей
худшая из топ-200 скважин во втором регионе даст 84.03888567782631 тыс баррелей
худшая из топ-200 скважин в третьем регионе даст 108.8077022306724 тыс баррелей


Похоже что не все скважины из топ-200 будут безубыточными. Найдем реальное количество безубыточных скважин.

In [15]:
print('в первом регионе необходимо взять', (100 - st.percentileofscore(true_predict_values_region_1['true'], 111.2)) * 5, 'скважин')
print('во втором регионе необходимо взять', (100 - st.percentileofscore(true_predict_values_region_2['true'], 111.2)) * 5, 'скважин')
print('в третьем регионе необходимо взять', (100 - st.percentileofscore(true_predict_values_region_3['true'], 111.2)) * 5, 'скважин')

в первом регионе необходимо взять 181.84 скважин
во втором регионе необходимо взять 82.5 скважин
в третьем регионе необходимо взять 190.36 скважин


Таким образом, я бы внес предложение о количестве разрабатываемых скважин в каждом регионе. Как видно, если брать 200, то некоторые скважины даже в "хороших" регионах будут убыточными. При этом даже во втором регионе "развернув" 82 скважины можно получить прибыль.

# 4. Риски и прибыль для каждого региона
<a id="4."></a>

## 4.1. Функции для проведения bootstrap и расчета прибыли
<a id="4.1."></a>

## 4.1.1. Функция расчета прибыли
<a id="4.1.1."></a>

Как мы определили в п.3.2 разработка одной скважины в среднем обходится 50 млн. руб. Тогда расчет прибыли можно провести двумя способами:
1. Провести расчет прибыли с каждоой скважины как  "объем запасов скважины" * 450.000 - 50.000.000. Затем просуммировать выбранные скважины.
2. Посчитать суммарное значение сырья как "суммарный объем запасов со всех скважин" * 450.000 - 10e9

Воспользуемся вторым способом:


In [16]:
'''
def profit_counter(product:pd.Series, count:int = 200, income:int = 450000):
    """
    функция возвращает суммарную прибыль для заданного "count" числа наибольших значений
    :param product: рассчитываемая выборка
    :param count: отсортированнаое по убыванию число скважин
    :param income: доход от одной скважины (р.)
    :return:
    """
    product_sorted = product.sort_values(ascending=False)
    top_count_product = product_sorted[:count]

    return int(top_count_product.sum()* income - 10e9)
'''

'\ndef profit_counter(product:pd.Series, count:int = 200, income:int = 450000):\n    """\n    функция возвращает суммарную прибыль для заданного "count" числа наибольших значений\n    :param product: рассчитываемая выборка\n    :param count: отсортированнаое по убыванию число скважин\n    :param income: доход от одной скважины (р.)\n    :return:\n    """\n    product_sorted = product.sort_values(ascending=False)\n    top_count_product = product_sorted[:count]\n\n    return int(top_count_product.sum()* income - 10e9)\n'

In [17]:
def profit_counter(true_subsample, predicted_subsample, count:int = 200, income:int = 450000):
    """
    функция сперва формирует лучшие "count" число (топ-200) из спрогнозированной выборки, 
    затем формирует выборку из соответствующиих им истинных значений
    и для каждой из полученных выборок (топ-200 спрогнозированных, топ-200 истинных) 
    возвращает суммарную прибыль 
    :param true_subsample: выборка истинных (500) значений
    :predicted_subsample: выборка спрогнозированных (500) значений
    :param count: отсортированное по убыванию число скважин
    :param income: доход от одной скважины (р.)
    :return:
    """
    
    predicted_sorted = predicted_subsample.sort_values(ascending=False)
    top_count_predicted = predicted_sorted[:count]   
    top_count_true = true_subsample[top_count_predicted.index.unique()]
    #print('top_count_true =', len(top_count_true))
    predicted_profit = int(top_count_predicted.sum()* income - 10e9)
    true_profit = int(top_count_true.sum()* income - 10e9)
    
    return true_profit, predicted_profit

## 4.1.2. Функция для проведения bootstrap
<a id="4.1.2."></a>

In [18]:
def bootstraper(series_true:pd.Series, series_predict:pd.Series, repeat:int=1000, sample_size:int = 500):
    """
    данная  функция:
     1. принимает на вход два столбца (формат Series) с реальными и спрогнозированными значеними объема запасов нефти
     2. заданное число раз формирует выборку заднного размера
     3. для каждой выборки вызывает функцию profit_counter, которая  расчитывает суммарную прибыль лучших 200 скважин
    :param series_true: реальные значения объёмов запасов в скважинах
    :param series_predict: спрогнозированные значения объёмов запасов в скважинах
    :param repeat: число выборок
    :param sample_size: размер выбокри
    :return:
    """
    profit_values_true = []
    profit_values_predicted = []
    state = np.random.RandomState(12345)
    for i in range(repeat):
        # отбираем sample_size (т.е. 500) скважин и сохраняем реальные ..
        true_subsample = series_true.sample(n=sample_size, replace=True, random_state=state)
        # .. и предсказанные значения
        predicted_subsample = series_predict[true_subsample.index]
        # расчитываем  реальную прибыль с топ-200
        #profit_true = profit_counter(true_subsample)
        # .. и предсказанную
        #profit_predicted = profit_counter(predicted_subsample)
        # добавляем в списки
        
        # новая строка кода
        profit_true, profit_predicted = profit_counter(true_subsample, predicted_subsample)
        
        profit_values_true.append(profit_true)
        profit_values_predicted.append(profit_predicted)
        # перенесем это в Series формат
    profit_values_true = pd.Series(profit_values_true)
    profit_values_predicted = pd.Series(profit_values_predicted)

    return profit_values_true, profit_values_predicted

## 4.2. Bootstrap
<a id="4.2."></a>

In [19]:
# суммарный реальный и спрогнозированный доход с лучших 200 из 500 скважин полученных бутстрепом
profit_values_true_1, profit_values_predicted_1 = bootstraper(true_predict_values_region_1['true'], true_predict_values_region_1['predicted'], repeat=2)
profit_values_true_2, profit_values_predicted_2 = bootstraper(true_predict_values_region_2['true'], true_predict_values_region_2['predicted'], repeat=1000)
profit_values_true_3, profit_values_predicted_3 = bootstraper(true_predict_values_region_3['true'], true_predict_values_region_3['predicted'], repeat=1000)

## 4.3. Средняя прибыль
<a id="4.3."></a>

In [20]:
print('средняя прибыль на основе реальных значений суммарной доходности для топ-200 скважин региона 1 составляет:', profit_values_true_1.mean())
print('средняя прибыль на основе спрогнозированных значений суммарной доходности для топ-200 скважин региона 1 составляет:', profit_values_predicted_1.mean())
print('_'*40)
print('средняя прибыль на основе реальных значений суммарной доходности для топ-200 скважин региона 2 составляет:', profit_values_true_2.mean())
print('средняя прибыль на основе спрогнозированных значений суммарной доходности для топ-200 скважин региона 2 составляет:', profit_values_predicted_2.mean())
print('_'*40)
print('средняя прибыль на основе реальных значений суммарной доходности для топ-200 скважин региона 3 составляет:', profit_values_true_3.mean())
print('средняя прибыль на основе спрогнозированных значений суммарной доходности для топ-200 скважин региона 3 составляет:', profit_values_predicted_3.mean())
print('_'*40)

средняя прибыль на основе реальных значений суммарной доходности для топ-200 скважин региона 1 составляет: 422620856.0
средняя прибыль на основе спрогнозированных значений суммарной доходности для топ-200 скважин региона 1 составляет: 141304058.5
________________________________________
средняя прибыль на основе реальных значений суммарной доходности для топ-200 скважин региона 2 составляет: 433149284.51
средняя прибыль на основе спрогнозированных значений суммарной доходности для топ-200 скважин региона 2 составляет: 430373476.863
________________________________________
средняя прибыль на основе реальных значений суммарной доходности для топ-200 скважин региона 3 составляет: 377557884.235
средняя прибыль на основе спрогнозированных значений суммарной доходности для топ-200 скважин региона 3 составляет: 266394474.666
________________________________________


## 4.4. Доверительный интервал
<a id="4.4."></a>

In [21]:
def confidence_interval_computer_old(df, alpha=0.95):
    interval_values = st.t.interval(alpha=alpha, df=len(df)-1, loc=df.mean(), scale=df.sem())
    return int(interval_values[0]), int(interval_values[1])

In [22]:
def confidence_interval_computer(df, alpha=0.95):
    interval_values_low = df.quantile(0.025)
    interval_values_up = df.quantile(0.975)
    return int(interval_values_low), int(interval_values_up)

In [23]:
print('95% доверительный интервал истинных значений суммарной прибыли для топ-200 скважин региона 1 составляет:', confidence_interval_computer(profit_values_true_1))
print('95% доверительный интервал спрогнозированных значений суммарной прибыли для топ-200 скважин региона 1 составляет:', confidence_interval_computer(profit_values_predicted_1))

95% доверительный интервал истинных значений суммарной прибыли для топ-200 скважин региона 1 составляет: (281719891, 563521820)
95% доверительный интервал спрогнозированных значений суммарной прибыли для топ-200 скважин региона 1 составляет: (6875756, 275732360)


In [24]:
print('95% доверительный интервал истинных значений суммарной прибыли для топ-200 скважин региона 1 составляет (предудущий вариант функции):', confidence_interval_computer_old(profit_values_true_1))
print('95% доверительный интервал спрогнозированных значений суммарной прибыли для топ-200 скважин региона 1 составляет (предудущий вариант функции):', confidence_interval_computer_old(profit_values_predicted_1))

95% доверительный интервал истинных значений суммарной прибыли для топ-200 скважин региона 1 составляет (предудущий вариант функции): (-1461922834, 2307164546)
95% доверительный интервал спрогнозированных значений суммарной прибыли для топ-200 скважин региона 1 составляет (предудущий вариант функции): (-1656668072, 1939276189)


In [25]:
print('95% доверительный интервал истинных значений суммарной прибыли для топ-200 скважин региона 2 составляет:', confidence_interval_computer(profit_values_true_2))
print('95% доверительный интервал спрогнозированных значений суммарной прибыли для топ-200 скважин региона 2 составляет:', confidence_interval_computer(profit_values_predicted_2))

95% доверительный интервал истинных значений суммарной прибыли для топ-200 скважин региона 2 составляет: (16846174, 815972525)
95% доверительный интервал спрогнозированных значений суммарной прибыли для топ-200 скважин региона 2 составляет: (17347696, 823513320)


In [26]:
print('95% доверительный интервал истинных значений суммарной прибыли для топ-200 скважин региона 3 составляет:', confidence_interval_computer(profit_values_true_3))
print('95% доверительный интервал спрогнозированных значений суммарной прибыли для топ-200 скважин региона 3 составляет:', confidence_interval_computer(profit_values_predicted_3))

95% доверительный интервал истинных значений суммарной прибыли для топ-200 скважин региона 3 составляет: (-170780417, 901772131)
95% доверительный интервал спрогнозированных значений суммарной прибыли для топ-200 скважин региона 3 составляет: (92354027, 465626446)


**Вывод:** получился такой рейтинг средних значений:
1. 2-й регион 639 млн
2. 1-й регион 615 млн
3. 3-й регион 587 млн

## 4.5. Вероятность убытка
<a id="4.5."></a>

Определим для себя что считать убытком: Убыток - это отрицательная прибыль. В связи с чем подготовим функцию расчета убытка.

In [27]:
def loss_probability_counter(df):
    """
    Функция оценивает полученную прибыль от топ-200 скважин по каждому из значений бутсреп-выборки и подсчитывает соотноешение убыточных вариантов топ-200 к общему числу выборки
    :param df:
    :param mean_value:
    :return:
    """
    counter = 0
    for i  in range(len(df)):
        if df[i] < 0:
            counter += 1
    return counter/len(df)

Вероятность убытка будем считать не по спрогнозированным значениям, а по реальным

In [28]:
print('вероятность убытка в регионе 1:', loss_probability_counter(profit_values_true_1))
print('вероятность убытка в регионе 2:', loss_probability_counter(profit_values_true_2))
print('вероятность убытка в регионе 3:', loss_probability_counter(profit_values_true_3))

вероятность убытка в регионе 1: 0.0
вероятность убытка в регионе 2: 0.019
вероятность убытка в регионе 3: 0.074


# 5. Выводы
<a id="5."></a>

Полученные данные по регионам сведены в таблицу 1.

| рейтинг   | регион | среднее значение | диапазон 95% возможных значений (доверитальный интервал) | вероятность убытка |
|---------|--------|------------------|----------------------------------------------------------|--------------------|
|1| 2-й    | 433 млн          | 16 - 815                                                 | 0%                 |
|2| 1-й    | 422 млн          | 281 - 563                                                | 1,9%               |
|3| 3-й    | 377  млн         | -170 - 901                                                | 7.4%               |

Требованию по вероятности убытков меньше 2.5% удовлетворяет только 2-й регион. Он же и лидер по прибыли. Он же и самый честный по предсказаниям.

**Выводы:**
1. Наибольшую прибыль с минимальными рисками можно получить в 2-м регионе
2. В качестве рекомендаций по увеличинию прибыли и снижению рисокв можно было бы предложить делать выборку не в 200 из 500 скважин, а 181 в первом, 82 - во втором, 190 - в третьем.